# Day 30: 最終プロジェクト（完成と展開）

## Learning Objectives
- モデルの最適化方法を学ぶ
- ONNX形式への変換を実装する
- Webアプリケーションの展開方法を理解する

## 1. モデルの最適化

In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import load_model
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import os
from pathlib import Path
from tensorflow_model_optimization.python import kernel_library as kl
from tensorflow_model_optimization.sparsity import keras as sparsity_params
from tensorflow_model_optimization.python.core import grappler \
  as tf_opt
import tempfile
import onnx
import onnxruntime as ort

### 1.1 量子化による最適化

In [ ]:
def create_simple_model():
    """テスト用の簡単なモデルを作成"""
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

def benchmark_model(model, test_data, model_name):
    """モデルのベンチマーク"""
    start_time = time.time()
    predictions = model.predict(test_data)
    inference_time = time.time() - start_time
    
    # モデルサイズの取得
    model_size = sum(np.prod(v.shape) for v in model.get_weights()) * 4 / 1024 / 1024  # MB
    
    # FLOPsの見積もり
    flops = model.count_params() * 2  # 粗略な見積もり
    
    return {
        'inference_time': inference_time,
        'fps': len(test_data) / inference_time,
        'model_size_mb': model_size,
        'flops': flops
    }

# 量子化手法の比較
def compare_quantization_methods():
    # テストデータの作成
    test_data = np.random.rand(10, 256, 256, 3).astype(np.float32)
    
    # オリジナルモデル
    model_original = create_simple_model()
    print("オリジナルモデルの作成完了")
    
    # ダミーの訓練データで学習（実際のアプリケーションでは実際のデータを使用）
    dummy_labels = np.random.randint(0, 10, (100,))
    model_original.fit(np.random.rand(100, 256, 256, 3).astype(np.float32), 
                      dummy_labels, epochs=2, verbose=0)
    
    # ベンチマーク
    print("\nオリジナルモデルのベンチマーク:")
    original_metrics = benchmark_model(model_original, test_data, "Original")
    print(f"推論時間: {original_metrics['inference_time']:.3f}s")
    print(f"FPS: {original_metrics['fps']:.1f}")
    print(f"モデルサイズ: {original_metrics['model_size_mb']:.2f}MB")
    
    # 1. ダイナミック量子化（Dynamic Quantization）
    print("\n--- ダイナミック量子化 ---")
    model_dynamic_quant = tfmot.quantization.keras.quantize_model(model_original)
    model_dynamic_quant.compile(optimizer='adam',
                              loss='sparse_categorical_crossentropy',
                              metrics=['accuracy'])
    
    dynamic_metrics = benchmark_model(model_dynamic_quant, test_data, "Dynamic")
    print(f"推論時間: {dynamic_metrics['inference_time']:.3f}s")
    print(f"FPS: {dynamic_metrics['fps']:.1f}")
    print(f"モデルサイズ: {dynamic_metrics['model_size_mb']:.2f}MB")
    
    # 2. 整数量子化（Integer Quantization）
    print("\n--- 整数量子化 ---")
    def representative_dataset():
        for data in test_data:
            yield [data]
    
    converter = tf.lite.TFLiteConverter.from_keras_model(model_original)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    
    tflite_quant_model = converter.convert()
    
    # TFLiteモデルのサイズ
    quant_size = len(tflite_quant_model) / 1024 / 1024  # MB
    print(f"量子化後のモデルサイズ: {quant_size:.2f}MB")
    print(f"サイズ削減率: {(1 - quant_size/original_metrics['model_size_mb'])*100:.1f}%")
    
    # 3. プルーニング（Pruning）
    print("\n--- プルーニング ---")
    
    def apply_pruning_to_conv(layer):
        pruning_params = sparsity_params.ConstantSparsity(0.5, begin_step=0, frequency=100)
        return kl.PruneLowMagnitude(layer, pruning_params=pruning_params)
    
    # プルーニングを適用したモデルの作成
    model_pruned = models.Sequential()
    model_pruned.add(apply_pruning_to_conv(layers.Conv2D(32, (3, 3), activation='relu', 
                                                       input_shape=(256, 256, 3))))
    model_pruned.add(layers.MaxPooling2D((2, 2)))
    model_pruned.add(layers.Flatten())
    model_pruned.add(apply_pruning_to_conv(layers.Dense(256, activation='relu')))
    model_pruned.add(apply_pruning_to_conv(layers.Dense(10, activation='softmax')))
    
    model_pruned.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
    
    # プルーニング訓練（簡略化）
    model_pruned.fit(np.random.rand(50, 256, 256, 3).astype(np.float32), 
                    dummy_labels[:50], epochs=1, verbose=0)
    
    pruned_metrics = benchmark_model(model_pruned, test_data, "Pruned")
    print(f"推論時間: {pruned_metrics['inference_time']:.3f}s")
    print(f"FPS: {pruned_metrics['fps']:.1f}")
    print(f"モデルサイズ: {pruned_metrics['model_size_mb']:.2f}MB")
    
    # 結果の比較
    methods = ['Original', 'Dynamic Quant', 'Integer Quant', 'Pruned']
    times = [original_metrics['inference_time'], 
            dynamic_metrics['inference_time'], 
            original_metrics['inference_time'],  # TFLiteは直接比較できないのでオリジナルを使用
            pruned_metrics['inference_time']]
    sizes = [original_metrics['model_size_mb'], 
             dynamic_metrics['model_size_mb'], 
             quant_size, 
             pruned_metrics['model_size_mb']]
    
    # 可視化
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # 推論時間の比較
    ax1.bar(methods, times, color=['blue', 'green', 'orange', 'red'])
    ax1.set_ylabel('Inference Time (s)')
    ax1.set_title('Inference Time Comparison')
    ax1.grid(True, alpha=0.3)
    
    # モデルサイズの比較
    ax2.bar(methods, sizes, color=['blue', 'green', 'orange', 'red'])
    ax2.set_ylabel('Model Size (MB)')
    ax2.set_title('Model Size Comparison')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'original': original_metrics,
        'dynamic_quant': dynamic_metrics,
        'integer_quant': {'model_size_mb': quant_size},
        'pruned': pruned_metrics
    }

# 量子化手法の比較を実行
optimization_results = compare_quantization_methods()

### 1.2 ONNX形式への変換

In [ ]:
def convert_to_onnx(model, input_shape=(1, 256, 256, 3)):
    """KerasモデルをONNXに変換"""
    try:
        # onnx-kerasのインストールが必要
        import onnxmltools as onnx
        
        # モデルをONNX形式に変換
        onnx_model = onnx.convert_keras(model)
        
        # 変換したモデルを保存
        onnx.save(onnx_model, "model.onnx")
        print("ONNXモデルの保存完了")
        
        # モデル情報の表示
        print("\nONNXモデル情報:")
        print(f"入力形状: {onnx_model.graph.input[0].type.tensor_type.shape}")
        print(f"出力形状: {onnx_model.graph.output[0].type.tensor_type.shape}")
        print(f"ノード数: {len(onnx_model.graph.node)}")
        
        return onnx_model
    except ImportError:
        print("onnxmltoolsがインストールされていません。")
        print("pip install onnxmltools でインストールしてください。")
        return None
    except Exception as e:
        print(f"ONNX変換中にエラーが発生しました: {e}")
        return None

def verify_onnx_model(onnx_model, test_data):
    """ONNXモデルの検証"""
    try:
        # ONNX Runtimeで推論
        ort_session = ort.InferenceSession(onnx_model.SerializeToString())
        
        # 入力名の取得
        input_name = ort_session.get_inputs()[0].name
        output_name = ort_session.get_outputs()[0].name
        
        # 推論実行
        start_time = time.time()
        ort_outputs = ort_session.run([output_name], {input_name: test_data.astype(np.float32)})
        ort_time = time.time() - start_time
        
        print(f"\nONNX Runtimeでの推論時間: {ort_time:.3f}s")
        print(f"FPS: {len(test_data) / ort_time:.1f}")
        
        # 出力形状の確認
        print(f"出力形状: {ort_outputs[0].shape}")
        
        # モデル情報の詳細表示
        print("\nONNXモデルの詳細:")
        print(f"プロバイダ: {ort_session.get_providers()}")
        print(f"入力名: {input_name}")
        print(f"出力名: {output_name}")
        
        return ort_session
    except Exception as e:
        print(f"ONNXモデルの検証中にエラーが発生しました: {e}")
        return None

def onnx_optimization_tips():
    """ONNXモデルの最適化方法のヒント"""
    tips = [
        {
            'name': '形状の固定',
            'description': 'モデルの入力形状を固定することで、最適化が効果的になります。',
            'code': 'model = ...  # モデル定義\n\n# ONNXに変換する前に入力形状を固定\ninput_shape = (1, 256, 256, 3)\ntest_input = np.random.rand(*input_shape).astype(np.float32)\n\n# モデルを固定形状で変換\nmodel.compile(...)\nmodel.predict(test_input)  # このとき形状が固定される'
        },
        {
            'name': 'Opsetバージョンの指定',
            'description': '最新のOpsetバージョンを使用することで、最適化を利用できます。',
            'code': 'onnx_model = onnx.convert_keras(model, \\n                    initializers=True,\
                    target_opset={"ai.onnx": 14})  # 適切なOpsetバージョンを指定'
        },
        {
            'name': '最適化の適用',
            'description': 'ONNX Optimizeを使用してモデルを最適化します。',
            'code': 'from onnxruntime.tools.optimizer import optimize\n\noptimized_model = optimize(model)\noptimized_model.save("optimized_model.onnx")'
        },
        {
            'name': '量化',
            'description': 'ONNX Quantizeを使用してモデルを量子化します。',
            'code': 'from onnxruntime.quantization import quantize_static\n\nquantize_static(model, "quantized_model.onnx", "calibration_data")'
        }
    ]
    
    print("=== ONNXモデル最適化のヒント ===")
    for i, tip in enumerate(tips, 1):
        print(f"\n{i}. {tip['name']}")
        print(f"   {tip['description']}")
        print("\nコード例:")
        print(f"```python\n{tip['code']}\n```")
    
    print("\n=== ONNX RuntimeでのGPU使用 ===")
    print("ONNX RuntimeはGPUをサポートしています。")
    print("GPUを使用するには、インストール時にCUDAを有効にしてください。")
    print("\nインストール方法:")
    print("```bash")
    print("pip install onnxruntime-gpu")
    print("```")
    
    print("\nGPUセッションの作成:")
    print("```python")
    print("ort_session = ort.InferenceSession(\"model.onnx\", providers=[\"CUDAExecutionProvider\"])\n")
    print("```")

# ONNX変換のテスト
print("ONNX形式への変換を試行...")
# create_simple_modelでモデルを作成
model = create_simple_model()
test_data = np.random.rand(1, 256, 256, 3).astype(np.float32)

# ダミーで予測を実行（実際の学習は不要）
model.predict(test_data, verbose=0)

# ONNXへの変換onnx_model = convert_to_onnx(model)

if onnx_model is not None:
    # ONNXモデルの検証
    ort_session = verify_onnx_model(onnx_model, test_data)
    # 最適化のヒントを表示
    onnx_optimization_tips()

## 2. Webアプリケーション展開

### 2.1 FlaskベースのWebアプリケーション

In [ ]:
app_code = '''
# app.py - Flask Webアプリケーション
from flask import Flask, request, jsonify, render_template, send_file
import numpy as np
import cv2
import io
import os
from PIL import Image
import json
from pathlib import Path
import base64
import time

# アプリケーションの初期化
app = Flask(__name__, 
            static_folder='static',
            template_folder='templates')

# グローバル変数
MODEL_LOADED = False
model = None
input_shape = (256, 256)

# メインページ
@app.route('/')
def index():
    return render_template('index.html')

# モデルのロード
@app.route('/api/load_model', methods=['POST'])
def load_model():
    global model, MODEL_LOADED
    
    try:
        # ここで実際のモデルロード処理を実装
        # 例: model = load_model('model.h5')
        
        MODEL_LOADED = True
        return jsonify({'status': 'success', 'message': 'モデルが正常にロードされました'})
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# 画像処理エンドポイント
@app.route('/api/process', methods=['POST'])
def process_image():
    if not MODEL_LOADED:
        return jsonify({'status': 'error', 'message': 'モデルがロードされていません'}), 400
    
    try:
        # リクエストから画像を取得
        if 'image' not in request.files:
            return jsonify({'status': 'error', 'message': '画像が送信されていません'}), 400
        
        file = request.files['image']
        if file.filename == '':
            return jsonify({'status': 'error', 'message': '画像が選択されていません'}), 400
        
        # 画像の読み込みと前処理
        image = Image.open(file.stream)
        image = image.convert('RGB')
        
        # 元画像の保存
        original_image = np.array(image)
        
        # 画像のリサイズ
        resized = image.resize(input_shape)
        processed_image = np.array(resized) / 255.0
        
        # ここで実際のモデル推論を実装
        # 例: prediction = model.predict(np.expand_dims(processed_image, axis=0))
        
        # ダミー処理（実際はモデルの予測結果）
        start_time = time.time()
        # 擬似的な処理遅延
        time.sleep(0.1)
        
        # ダミー結果の生成
        detection_results = []
        num_detections = np.random.randint(1, 5)
        
        for i in range(num_detections):
            x = np.random.randint(0, input_shape[0] - 50)
            y = np.random.randint(0, input_shape[1] - 50)
            w = np.random.randint(30, 100)
            h = np.random.randint(30, 100)
            confidence = np.random.uniform(0.7, 0.95)
            class_id = np.random.randint(0, 10)
            
            detection_results.append({
                'box': [x, y, w, h],
                'confidence': confidence,
                'class': f'object_{class_id}'
            })
        
        processing_time = time.time() - start_time
        
        # 結果画像の作成
        result_image = original_image.copy()
        
        # バウンディングボックスの描画
        colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), 
                  (255, 255, 0), (255, 0, 255)]
        
        for det in detection_results:
            x, y, w, h = det['box']
            color = colors[len(detection_results) % len(colors)]
            
            # バウンディングボックス
            cv2.rectangle(result_image, (x, y), (x + w, y + h), color, 2)
            
            # ラベル
            label = f"{det['class']}: {det['confidence']:.2f}"
            cv2.putText(result_image, label, (x, y - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # 結果画像をバイトに変換
        result_pil = Image.fromarray(result_image)
        img_byte_arr = io.BytesIO()
        result_pil.save(img_byte_arr, format='JPEG')
        img_byte_arr.seek(0)
        
        # レスポンスの作成
        response = {
            'status': 'success',
            'processing_time': processing_time,
            'detections': detection_results,
            'image_base64': base64.b64encode(img_byte_arr.getvalue()).decode('utf-8')
        }
        
        return jsonify(response)
        
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# バッチ処理エンドポイント
@app.route('/api/batch_process', methods=['POST'])
def batch_process():
    if not MODEL_LOADED:
        return jsonify({'status': 'error', 'message': 'モデルがロードされていません'}), 400
    
    try:
        # ファイルの取得
        files = request.files.getlist('images')
        if not files:
            return jsonify({'status': 'error', 'message': '画像が送信されていません'}), 400
        
        results = []
        start_time = time.time()
        
        for file in files:
            # 各画像の処理
            image = Image.open(file.stream)
            image = image.convert('RGB')
            
            # ダミー処理
            result = {
                'filename': file.filename,
                'detections': [{'class': 'sample', 'confidence': 0.85}],
                'processing_time': np.random.uniform(0.05, 0.15)
            }
            
            results.append(result)
        
        total_time = time.time() - start_time
        
        return jsonify({
            'status': 'success',
            'total_time': total_time,
            'results': results
        })
        
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# モデル情報の取得
@app.route('/api/model_info', methods=['GET'])
def model_info():
    info = {
        'loaded': MODEL_LOADED,
        'input_shape': input_shape,
        'version': '1.0.0'
    }
    return jsonify(info)

# エラーハンドリング
@app.errorhandler(404)
def not_found(error):
    return jsonify({'status': 'error', 'message': 'エンドポイントが見つかりません'}), 404

@app.errorhandler(500)
def internal_error(error):
    return jsonify({'status': 'error', 'message': '内部サーバーエラー'}), 500

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
'''

# テンプレートファイルの作成
template_code = '''
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>物体検出Webアプリ</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body {
            background-color: #f8f9fa;
        }
        .card {
            margin-top: 20px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        }
        .preview-container {
            position: relative;
            max-width: 100%;
            margin: 20px 0;
        }
        .preview-image {
            max-width: 100%;
            height: auto;
            border: 2px solid #dee2e6;
            border-radius: 4px;
        }
        .loading {
            display: none;
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background: rgba(255, 255, 255, 0.8);
            justify-content: center;
            align-items: center;
        }
        .detection-item {
            padding: 10px;
            margin: 5px 0;
            background: #e9ecef;
            border-radius: 4px;
        }
    </style>
</head>
<body>
    <div class="container mt-4">
        <div class="row">
            <div class="col-md-12">
                <div class="card">
                    <div class="card-header bg-primary text-white">
                        <h1 class="mb-0">物体検出Webアプリ</h1>
                    </div>
                    <div class="card-body">
                        <!-- モデルロードボタン -->
                        <div class="mb-3">
                            <button id="loadModelBtn" class="btn btn-success">モデルをロード</button>
                            <span id="modelStatus" class="ms-2"></span>
                        </div>
                        
                        <!-- 画像アップロード -->
                        <div class="mb-3">
                            <h5>画像アップロード</h5>
                            <input type="file" id="imageInput" accept="image/*" class="form-control">
                        </div>
                        
                        <!-- 処理ボタン -->
                        <div class="mb-3">
                            <button id="processBtn" class="btn btn-primary" disabled>画像を処理</button>
                        </div>
                        
                        <!-- プレビュー領域 -->
                        <div class="row">
                            <div class="col-md-6">
                                <h5>元の画像</h5>
                                <div class="preview-container">
                                    <img id="originalImage" class="preview-image" style="display: none;">
                                    <div id="loadingOriginal" class="loading">
                                        <div class="spinner-border text-primary" role="status">
                                            <span class="visually-hidden">Loading...</span>
                                        </div>
                                    </div>
                                </div>
                            </div>
                            <div class="col-md-6">
                                <h5>検出結果</h5>
                                <div class="preview-container">
                                    <img id="resultImage" class="preview-image" style="display: none;">
                                    <div id="loadingResult" class="loading">
                                        <div class="spinner-border text-success" role="status">
                                            <span class="visually-hidden">Processing...</span>
                                        </div>
                                    </div>
                                </div>
                            </div>
                        </div>
                        
                        <!-- 結果表示 -->
                        <div id="resultsSection" class="mt-4" style="display: none;">
                            <h5>検出結果</h5>
                            <div id="detectionResults"></div>
                            <div class="mt-3">
                                <small class="text-muted">処理時間: <span id="processingTime">0</span> ms</small>
                            </div>
                        </div>
                    </div>
                </div>
            </div>
        </div>
    </div>
    
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/js/bootstrap.bundle.min.js"></script>
    <script>
        // DOM要素
        const loadImageBtn = document.getElementById('loadModelBtn');
        const modelStatus = document.getElementById('modelStatus');
        const imageInput = document.getElementById('imageInput');
        const processBtn = document.getElementById('processBtn');
        const originalImage = document.getElementById('originalImage');
        const resultImage = document.getElementById('resultImage');
        const resultsSection = document.getElementById('resultsSection');
        const detectionResults = document.getElementById('detectionResults');
        const processingTime = document.getElementById('processingTime');
        
        let modelLoaded = false;
        
        // モデルロード
        loadImageBtn.addEventListener('click', async () => {
            loadImageBtn.disabled = true;
            loadImageBtn.innerHTML = '<span class="spinner-border spinner-border-sm" role="status" aria-hidden="true"></span> Loading...';
            
            try {
                const response = await fetch('/api/load_model', { method: 'POST' });
                const data = await response.json();
                
                if (data.status === 'success') {
                    modelLoaded = true;
                    modelStatus.textContent = 'モデルがロードされました';
                    modelStatus.className = 'ms-2 text-success';
                    processBtn.disabled = false;
                } else {
                    throw new Error(data.message);
                }
            } catch (error) {
                modelStatus.textContent = 'エラー: ' + error.message;
                modelStatus.className = 'ms-2 text-danger';
            } finally {
                loadImageBtn.disabled = false;
                loadImageBtn.innerHTML = 'モデルをロード';
            }
        });
        
        // 画像プレビュー
        imageInput.addEventListener('change', (e) => {
            const file = e.target.files[0];
            if (file) {
                const reader = new FileReader();
                reader.onload = (e) => {
                    originalImage.src = e.target.result;
                    originalImage.style.display = 'block';
                    processBtn.disabled = !modelLoaded;
                };
                reader.readAsDataURL(file);
            }
        });
        
        // 画像処理
        processBtn.addEventListener('click', async () => {
            if (!modelLoaded || !imageInput.files[0]) {
                return;
            }
            
            processBtn.disabled = true;
            processBtn.innerHTML = '<span class="spinner-border spinner-border-sm" role="status" aria-hidden="true"></span> Processing...';
            
            // ローディング表示
            document.getElementById('loadingOriginal').style.display = 'flex';
            document.getElementById('loadingResult').style.display = 'flex';
            
            const formData = new FormData();
            formData.append('image', imageInput.files[0]);
            
            try {
                const startTime = performance.now();
                const response = await fetch('/api/process', {
                    method: 'POST',
                    body: formData
                });
                const data = await response.json();
                const endTime = performance.now();
                
                if (data.status === 'success') {
                    // 結果画像の表示
                    resultImage.src = 'data:image/jpeg;base64,' + data.image_base64;
                    resultImage.style.display = 'block';
                    
                    // 検出結果の表示
                    detectionResults.innerHTML = '';
                    data.detections.forEach(det => {
                        const item = document.createElement('div');
                        item.className = 'detection-item';
                        item.innerHTML = `
                            <strong>${det.class}</strong><br>
                            確信度: ${(det.confidence * 100).toFixed(1)}%<br>
                            位置: [${det.box[0]}, ${det.box[1]}] サイズ: ${det.box[2]}x${det.box[3]}
                        `;
                        detectionResults.appendChild(item);
                    });
                    
                    // 結果セクションの表示
                    resultsSection.style.display = 'block';
                    processingTime.textContent = data.processing_time.toFixed(2);
                    
                } else {
                    throw new Error(data.message);
                }
            } catch (error) {
                alert('処理中にエラーが発生しました: ' + error.message);
            } finally {
                // ローディング非表示
                document.getElementById('loadingOriginal').style.display = 'none';
                document.getElementById('loadingResult').style.display = 'none';
                
                processBtn.disabled = false;
                processBtn.innerHTML = '画像を処理';
            }
        });
    </script>
</body>
</html>
'''

# ディレクトリ構造の作成
def create_web_app_structure():
    """Webアプリのディレクトリ構造を作成"""
    dirs = ['app', 'app/static', 'app/templates', 'app/uploads', 'app/results']
    
    for dir_name in dirs:
        os.makedirs(dir_name, exist_ok=True)
    
    # アプリケーションファイルの作成
    with open('app/app.py', 'w', encoding='utf-8') as f:
        f.write(app_code)
    
    # テンプレートファイルの作成
    with open('app/templates/index.html', 'w', encoding='utf-8') as f:
        f.write(template_code)
    
    # requirements.txtの作成
    requirements = '''
flask==2.0.1
numpy==1.21.0
Pillow==8.3.1
opencv-python==4.5.3.56
'''
    
    with open('app/requirements.txt', 'w') as f:
        f.write(requirements)
    
    # READMEの作成
    readme = '''
# 物体検出Webアプリ

Flaskベースの物体検出Webアプリケーションです。

## 機能
- 画像のアップロード
- 物体検出
- 結果の可視化
- バッチ処理サポート

## 実行方法

### 1. 依存関係のインストール
```bash
cd app
pip install -r requirements.txt
```

### 2. アプリケーションの実行
```bash
python app.py
```

### 3. アクセス
ブラウザで http://localhost:5000 にアクセス

## ディレクトリ構造
```
app/
├── app.py              # メインアプリケーション
├── requirements.txt     # 依存関係
├── static/             # 静的ファイル
├── templates/          # HTMLテンプレート
│   └── index.html
├── uploads/            # アップロードされたファイル
└── results/            # 処理結果
```
'''
    
    with open('app/README.md', 'w', encoding='utf-8') as f:
        f.write(readme)
    
    print("Webアプリケーションの構造を作成しました")
    print("場所: app/ ディレクトリ")
    print("\n実行手順:")
    print("1. cd app")
    print("2. pip install -r requirements.txt")
    print("3. python app.py")

# Webアプリ構造の作成
create_web_app_structure()

### 2.2 Dockerコンテナ化

In [ ]:
# Dockerfileの作成
dockerfile_content = '''
# Dockerfile
FROM python:3.9-slim

WORKDIR /app

# 依存関係のインストールRUN apt-get update && apt-get install -y 
    libgl1-mesa-glx \\
    libglib2.0-0 \\
    libsm6 \\
    libxext6 \\
    libxrender-dev \\
    && rm -rf /var/lib/apt/lists/*

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

# ポートの公開
EXPOSE 5000

# 環境変数の設定
ENV FLASK_ENV=production

# アプリケーションの実行
CMD ["python", "app.py"]
'''

# docker-compose.ymlの作成
compose_content = '''
version: '3.8'

services:
  webapp:
    build: .
    ports:
      - "5000:5000"
    volumes:
      - ./app:/app
      - ./data:/app/data
      - ./models:/app/models
    environment:
      - FLASK_ENV=production
    restart: unless-stopped

  nginx:
    image: nginx:alpine
    ports:
      - "80:80"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf
      - ./app:/usr/share/nginx/html
    depends_on:
      - webapp
    restart: unless-stopped
'''

# nginx.confの作成
nginx_conf = '''
events {
    worker_connections 1024;
}

http {
    upstream webapp {
        server webapp:5000;
    }

    server {
        listen 80;
        server_name localhost;

        location / {
            proxy_pass http://webapp;
            proxy_set_header Host $host;
            proxy_set_header X-Real-IP $remote_addr;
            proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
            proxy_set_header X-Forwarded-Proto $scheme;
        }

        # 静的ファイルの直接提供
        location /static/ {
            alias /usr/share/nginx/html/static/;
            expires 1y;
            add_header Cache-Control "public, immutable";
        }

        # 動的コンテンツのキャッシュ設定
        location ~* \.(jpg|jpeg|png|gif|ico|css|js)$ {
            proxy_pass http://webapp;
            expires 1d;
            add_header Cache-Control "public";
        }
    }
}
'''

# Docker関連ファイルの作成
with open('app/Dockerfile', 'w') as f:
    f.write(dockerfile_content)

with open('app/docker-compose.yml', 'w') as f:
    f.write(compose_content)

with open('app/nginx.conf', 'w') as f:
    f.write(nginx_conf)

# Docker実行スクリプトの作成
docker_run_script = '''
#!/bin/bash

# Dockerビルド
echo "Dockerイメージをビルド中..."
docker build -t object-detection-webapp .

# コンテナの実行
echo "コンテナを実行中..."
docker run -d \\
    --name object-detection \\
    -p 5000:5000 \\
    -v $(pwd)/app:/app \\
    -v $(pwd)/data:/app/data \\
    -v $(pwd)/models:/app/models \\
    object-detection-webapp

# 実行状態の確認
echo "コンテナの状態:"
docker ps | grep object-detection

echo "アプリケーションは http://localhost:5000 でアクセスできます"


### 2.3 FastAPIでの高性能展開


In [ ]:
# FastAPIアプリケーションの例
fastapi_code = '''
# main.py - FastAPIアプリケーション
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import JSONResponse, HTMLResponse
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
import numpy as np
import cv2
import io
import base64
import time
import uvicorn
from typing import List

# アプリケーションの初期化
app = FastAPI(
    title="物体検出API",
    description="高性能物体検出API",
    version="1.0.0"
)

# CORSの設定
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 静的ファイルのマウント
app.mount("/static", StaticFiles(directory="static"), name="static")

@app.get("/")
async def read_root():
    return {"message": "物体検出APIへようこそ"}

@app.get("/docs")
async def docs():
    return HTMLResponse("<html>
    <head>
        <title>物体検出API</title>
    </head>
    <body>
        <h1>物体検出API</h1>
        <p>/docs でAPIドキュメントを確認できます</p>
    </body>
</html>)

@app.get("/health")
async def health_check():
    return {"status": "healthy"}

@app.post("/predict")
async def predict_image(file: UploadFile = File(...)):
    """単一画像の物体検出"""
    try:
        # 画像の読み込み
        contents = await file.read()
        nparr = np.frombuffer(contents, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        
        if image is None:
            raise HTTPException(status_code=400, detail="画像の読み込みに失敗しました")
        
        # 画像のリサイズ
        resized = cv2.resize(image, (256, 256))
        
        # ダミー処理（実際はモデル推論）
        start_time = time.time()
        time.sleep(0.05)  # 擬似的な処理時間
        
        # ダミー検出結果
        detections = []
        for i in range(np.random.randint(1, 4)):
            x = np.random.randint(0, 200)
            y = np.random.randint(0, 200)
            w = np.random.randint(30, 80)
            h = np.random.randint(30, 80)
            conf = np.random.uniform(0.7, 0.95)
            cls = f"object_{np.random.randint(0, 10)}"
            
            detections.append({
                "box": [x, y, w, h],
                "confidence": conf,
                "class": cls
            })
        
        # 処理時間
        processing_time = time.time() - start_time
        
        # 結果画像の作成
        result_image = image.copy()
        colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
        
        for det in detections:
            x, y, w, h = det["box"]
            color = colors[len(detections) % len(colors)]
            cv2.rectangle(result_image, (x, y), (x + w, y + h), color, 2)
            label = f"{det['class']}: {det['confidence']:.2f}"
            cv2.putText(result_image, label, (x, y - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # 結果画像をBase64に変換
        _, buffer = cv2.imencode('.jpg', result_image)
        result_base64 = base64.b64encode(buffer).decode('utf-8')
        
        return JSONResponse({
            "success": True,
            "processing_time": processing_time,
            "detections": detections,
            "result_image": result_base64
        })
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/batch_predict")
async def batch_predict(files: List[UploadFile] = File(...)):
    """複数画像の一括処理"""
    if len(files) > 10:
        raise HTTPException(status_code=400, detail="一度に処理できる画像は10枚までです")
    
    results = []
    start_time = time.time()
    
    for file in files:
        try:
            # 画像の読み込み
            contents = await file.read()
            nparr = np.frombuffer(contents, np.uint8)
            image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            
            if image is None:
                results.append({
                    "filename": file.filename,
                    "error": "画像の読み込みに失敗しました"
                })
                continue
            
            # ダミー処理
            time.sleep(0.03)  # 擬似的な処理時間
            
            results.append({
                "filename": file.filename,
                "detections": [{"class": "sample", "confidence": 0.85}],
                "processing_time": 0.03
            })
            
        except Exception as e:
            results.append({
                "filename": file.filename,
                "error": str(e)
            })
    
    total_time = time.time() - start_time
    
    return JSONResponse({
        "success": True,
        "total_time": total_time,
        "results": results
    })

@app.get("/model/info")
async def model_info():
    """モデル情報の取得"""
    return {
        "model_name": "物体検出モデル",
        "version": "1.0.0",
        "input_size": (256, 256),
        "classes": 10
    }

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

# FastAPI関連ファイルの作成
with open('app/fastapi_main.py', 'w', encoding='utf-8') as f:
    f.write(fastapi_code)

# FastAPI用のrequirements.txt
fastapi_requirements = '''
fastapi==0.68.0
uvicorn==0.15.0
python-multipart==0.0.5
numpy==1.21.0
opencv-python==4.5.3.56
Pillow==8.3.1
'''

with open('app/fastapi_requirements.txt', 'w') as f:
    f.write(fastapi_requirements)

print("\n=== FastAPIアプリケーションを作成しました ===")
print("場所: app/fastapi_main.py")
print("\n実行手順:")
print("1. cd app")
print("2. pip install -r fastapi_requirements.txt")
print("3. python fastapi_main.py")
print("4. http://localhost:8000/docs でAPIドキュメントを確認")


## 3. デプロイ戦略

In [ ]:
# デプロイ戦略のまとめ
deployment_strategies = {
    "local_deployment": {
        "title": "ローカルデプロイ",
        "description": "開発環境でのテスト",
        "pros": ["手軽", "すぐに実行可能", "デバッグしやすい"],
        "cons": ["スケーラビリティなし", "可用性低い", "本番環境非対応"],
        "setup": "
1. pip install -r requirements.txt
2. python app.py または fastapi_main.py
3. ブラウザでアクセス
        ""
    },
    "docker_deployment": {
        "title": "Dockerコンテナ",
        "description": "環境の再現性が高い", 
        "pros": ["環境の再現性", "移植性が高い", "リソース効率が良い"],
        "cons": ["イメージサイズが大きい", "コンテナ管理が必要", "学習曲線"],
        "setup": "
1. Dockerfileの作成
2. docker build -t app .
3. docker run -p 5000:5000 app
        ""
    },
    "cloud_deployment": {
        "title": "クラウドサービス",
        "description": "本番環境向け", 
        "pros": ["スケーラビリティ", "高可用性", "管理しやすい"],
        "cons": ["コスト", "設定が複雑", "依存関係"],
        "setup": "
AWS:
1. ECRにイメージをプッシュ
2. ECS/EKSでデプロイ
3. ALBでロードバランシング

GCP:
1. Container Registryにプッシュ
2. GKEでデプロイ
3. Cloud Load Balancing

Azure:
1. Container Registryにプッシュ
2. AKSでデプロイ
3. Application Gateway
        ""
    },
    "serverless_deployment": {
        "title": "サーバーレス", 
        "description": "イベント駆動", 
        "pros": ["コスト効率が良い", "スケール容易", "管理不要"],
        "cons": ["実行時間制限", "コールドスタート", "デバッグが難しい"],
        "setup": "
AWS Lambda:
1. Lambda関数を作成
2. S3トリガーを設定
3. API Gatewayで公開

Google Cloud Functions:
1. 関数を作成
2. Cloud Storageトリガー
3. Cloud Functions API
        ""
    }
}

# デプロイ戦略の表示
print("=== デプロイ戦略の比較 ===")
for strategy, info in deployment_strategies.items():
    print(f"\n{strategy.upper()}: {info['title']}")
    print(f"説明: {info['description']}")
    print("\nメリット:")
    for pro in info['pros']:
        print(f"  - {pro}")
    print("\nデメリット:")
    for con in info['cons']:
        print(f"  - {con}")
    print("\nセットアップ:")
    print(info['setup'])
    print("\n" + "="*50 + "\n")

# プロダクションチェックリスト
production_checklist = [
    {
        "category": "セキュリティ",
        "items": [
            "HTTPSの使用",
            "認証と認可の実装",
            "入力バリデーション",
            "リクエストサイズ制限",
            "APIキーの管理",
            "セキュアなヘッダーの設定"
        ]
    },
    {
        "category": "パフォーマンス",
        "items": [
            "モデルの最適化",
            "キャッシュ戦略の実装",
            "非同期処理の使用",
            "接続プールの設定",
            "CDNの利用",
            "圧縮の有効化"
        ]
    },
    {
        "category": "監視とログ",
        "items": [
            "エラーロギング",
            "パフォーマンスモニタリング",
            "アラート設定",
            "リソース使用量の監視",
            "ビジネス指標の追跡",
            "ログの集約と分析"
        ]
    },
    {
        "category": "バックアップとリカバリ",
        "items": [
            "定期的なバックアップ",
            "バージョン管理",
            "災害復旧計画",
            "データ復元のテスト",
            "マルチリージョン配置",
            "自動スケーリングの設定"
        ]
    }
]

print("=== プロダクション展開チェックリスト ===")
for section in production_checklist:
    print(f"\n[{section['category']}]")
    for item in section['items']:
        print(f"  [ ] {item}")
    print()

# CI/CDパイプラインの例
cicd_pipeline = '''
# .gitlab-ci.yml - CI/CDパイプラインの例
stages:
  - test
  - build
  - deploy

variables:
  DOCKER_DRIVER: overlay2
  DOCKER_TLS_CERTDIR: "/certs"

cache:
  paths:
    - .venv/
    - node_modules/

# テストステージ
unit_test:
  stage: test
  image: python:3.9
  before_script:
    - python -m venv .venv
    - source .venv/bin/activate
    - pip install -r requirements.txt
  script:
    - pytest tests/ -v
    - flake8 app/
    - black --check app/

# ビルドステージ
build_docker:
  stage: build
  image: docker:latest
  services:
    - docker:dind
  script:
    - docker build -t object-detection:$CI_COMMIT_SHA .
    - docker push $CI_REGISTRY/object-detection:$CI_COMMIT_SHA
  only:
    - main

# デプロイステージ
deploy_staging:
  stage: deploy
  image: alpine/k8s:latest
  script:
    - kubectl config use-context staging
    - kubectl set image deployment/object-detection app=object-detection:$CI_COMMIT_SHA
    - kubectl rollout status deployment/object-detection
  only:
    - main

deploy_production:
  stage: deploy
  image: alpine/k8s:latest
  script:
    - kubectl config use-context production
    - kubectl set image deployment/object-detection app=object-detection:$CI_COMMIT_SHA
    - kubectl rollout status deployment/object-detection
    - kubectl rollout restart deployment/monitoring
  when: manual
  only:
    - main
'''

print("\n=== CI/CDパイプライン例 ===")
print(cicd_pipeline)

## Week 6 振り返り

### 習得したスキル:
- [x] モデル量子化の方法を理解した
- [x] ONNX形式への変換を実装した
- [x] FlaskベースのWebアプリを作成した
- [x] Dockerコンテナ化を実装した
- [x] FastAPIでの高性能展開を理解した
- [x] デプロイ戦略を学んだ
- [x] CI/CDパイプラインの概念を理解した

**お疲れ様でした！** 全プロジェクトの完成です！

### 今後の発展方向:
- さらに大規模なモデルの最適化
- エッジデバイスへの展開
- リアルタイム処理の高速化
- マルチモーダルなモデル
- フェデレーテッドラーニング
- MLOpsの自動化